In [1]:
import pandas  as pd

dataset = pd.read_csv('../ozhegov_dataset.csv')
dataset = dataset[dataset['title'].str.len() > 0]
dataset = dataset[dataset['text'].str.len() > 0]
#для lenta - подсовываем даты
#dataset['title'] = dataset['date']
dataset = dataset.sample(n=1000)

dataset.astype({"text": str, "title": str})
dataset.info(show_counts=True)

dataset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 4186 to 16466
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   title       1000 non-null   object
 2   text        1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


,Unnamed: 0,title,text
4186,4186,ГАБАРДИН,Плотная шерстяная ткань с мелкими наклонными р...
23503,23503,РЕНКЛОД,Сорт крупной сладкой сливы. II прил. ренклбдны...
9798,9798,КНУТОВИЩЕ,Рукоятка кнута.
15543,15543,ОДЕКОЛОН,Гигиеническое и освежающее средство - спиртово...
8987,8987,ИТАК,"Следовательно, таким образом. Я, вопрос решен."


In [2]:
%pip install -U sentence-transformers ipywidgets weaviate-client chardet charset-normalizer

   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   ----------- ---------------------------- 51.2/171.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 171.5/171.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ---------------------------------------- 307.7/307.7 kB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/223.7 kB ? eta -:--:--
   --------------------------------------- 223.7/223.7 kB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.9 MB 17.4 MB/s eta 0:00:01
   ------------------------- -------------- 2.5/3.9 MB 26.5 MB/s eta 0:00:01
   ---------------------------------------- 3.9/3.9 MB 31.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 33.5 MB/s eta 0:00:00
   -------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
opentelemetry-proto 1.23.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.26.1 which is incompatible.


In [3]:
from weaviate.classes.config import VectorDistances
models = [
    "intfloat/multilingual-e5-large",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli",
    "cointegrated/LaBSE-en-ru",
    "sentence-transformers/LaBSE"
]

distances = [
    VectorDistances.L2_SQUARED,
    VectorDistances.DOT,
    VectorDistances.COSINE
]

In [38]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout


client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051
)


In [41]:
from sentence_transformers import SentenceTransformer

def text2vec(model_name, text):
    model = SentenceTransformer(model_name)
    embedding = model.encode(text)

    return embedding

text2vec("cointegrated/LaBSE-en-ru", "тетстстстс")

array([ 4.54010032e-02, -7.34052956e-02,  4.04356420e-02, -1.21046221e-02,
        2.86439769e-02,  3.78062017e-02, -9.19423159e-03,  1.12835374e-02,
       -6.88639060e-02, -3.03849112e-02,  3.47262993e-02,  1.35816494e-02,
        7.81860389e-03,  7.30751408e-03,  1.52076466e-03, -7.95842260e-02,
        2.60597561e-02, -3.04815043e-02,  3.19397822e-02,  2.87249740e-02,
       -5.07127028e-03,  8.36116634e-03,  5.07940911e-03, -4.79265563e-02,
       -1.53173264e-02, -3.07753459e-02, -3.82001251e-02,  3.80781926e-02,
       -4.12984379e-02,  5.59009276e-02, -2.93331947e-02, -1.77094769e-02,
       -4.30827141e-02, -2.97204917e-03, -3.62913050e-02, -6.67910129e-02,
       -3.13791223e-02,  5.87444333e-03, -1.06990021e-02, -7.24172369e-02,
        3.48523632e-02,  9.07385431e-04, -6.61408994e-03, -1.12753548e-02,
        4.88761328e-02,  2.36134306e-02,  1.83127020e-02, -1.77689511e-02,
        1.53741159e-03, -1.60640124e-02, -4.46677115e-03, -3.42552923e-02,
       -6.87659308e-02, -

In [57]:
from tqdm import tqdm 
from weaviate.classes.config import Configure, Property, DataType, Tokenization, VectorDistances

def create_collection(client, model_name, distance):
    
    client.collections.create(
        "title",
        properties=[ # properties configuration is optional
            Property(name="title", data_type=DataType.TEXT),
        ],
        vectorizer_config=[
            Configure.NamedVectors.none( 
                name="title", vector_index_config=Configure.VectorIndex.hnsw(distance_metric=distance)
            ),
        ]
    )

    collection = client.collections.get("title")
    
    with collection.batch.fixed_size(batch_size=200) as batch:
        for i, data in tqdm(dataset.iterrows()):

            obj = {
                "title": data["title"]
            }

            # Add object to batch queue
            batch.add_object(
                properties = obj,
                vector={
                "title": text2vec(model_name, data["title"]),
            }
            )

    # if len(collection.batch.failed_objects) > 0:
    #     print(collection.batch.failed_objects)

def delete_collection(client):
    client.collections.delete("title")


def query_collection(collection, query, max_results, dataframe, model_name, distance):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    #print(results)
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':list(map(float,results['distances'][0])),
                'query': query,
                'title': dataframe[dataframe.index.isin(list(map(int,results['ids'][0])))]['title'],
                'content': dataframe[dataframe.index.isin(list(map(int,results['ids'][0])))]['text'],
                'model_name': model_name,
                'distance': distance
                })
    
    # Забираем с минимальной дистанцией, значит он ближе и больше похож
    df = df[df.score == df.score.min()]
    df['is_found'] = df.apply(lambda row: row.query == row.title, axis=1)
    
    return df

In [58]:
delete_collection(client)
create_collection(client, "cointegrated/LaBSE-en-ru", VectorDistances.COSINE)

1it [00:02,  2.77s/it]Exception in callback PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()
handle: <Handle PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "src\\python\\grpcio\\grpc\\_cython\\_cygrpc/aio/completion_queue.pyx.pxi", line 147, in grpc._cython.cygrpc.PollerCompletionQueue._handle_events
BlockingIOError: [WinError 10035] A non-blocking socket operation could not be completed immediately
10it [00:21,  2.15s/it]Exception in callback PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()
handle: <Handle PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.1

KeyboardInterrupt: 

In [56]:
collection = client.collections.get("title")
for item in collection.iterator():
    print(item.properties)
    print(item.vector)

C:\Users\golov\AppData\Local\Temp\ipykernel_17328\1469566424.py:1: ResourceWarning: unclosed <socket.socket fd=3548, family=2, type=1, proto=0, laddr=('127.0.0.1', 56104), raddr=('127.0.0.1', 8080)>
  collection = client.collections.get("title")


{'title': 'КАПЛУН'}
{}
{'title': 'ОТКЛЕВАТЬ'}
{}
{'title': 'ОТСВЕЧИВАТЬ'}
{}
{'title': 'ТАНТАЛ'}
{}
{'title': 'РАЗМОКРОПОГОДИТЬСЯ'}
{}
{'title': 'ОБЩЕСТВЕННЫЙ'}
{}
{'title': 'ПОТВОРСТВОВАТЬ'}
{}
{'title': 'ВИДЕНИЕ'}
{}
{'title': 'СМОКВА'}
{}
{'title': 'ПЛЕМЯШ'}
{}
{'title': 'ВОЗБУДИМЫЙ'}
{}
{'title': 'РВАНЫЙ'}
{}
{'title': 'ЯДОХИМИКАТЫ'}
{}
{'title': 'БОЛТОВНЯ'}
{}
{'title': 'МОЗГЛЯВЫЙ'}
{}
{'title': 'ОБСТРЕЛ'}
{}
{'title': 'ОБЕСПОКОИТЬСЯ'}
{}
{'title': 'МИРНЫЙ'}
{}
{'title': 'КОЛКИЙ'}
{}
{'title': 'ПОСТИЛКА'}
{}
{'title': 'КАМЕНЕТЬ'}
{}
{'title': 'РАСКОВАННЫЙ'}
{}
{'title': 'НОЖИК'}
{}
{'title': 'ПРИРОДНЫЙ'}
{}
{'title': 'МОЛОКО'}
{}
{'title': 'СМЕШАННЫЙ'}
{}
{'title': 'ПОПУСТИТЕЛЬ'}
{}
{'title': 'КИНОВЕД'}
{}
{'title': 'ХРУЩ'}
{}
{'title': 'ПОДРЕЗАТЬ'}
{}
{'title': 'ПЕРЕПЕЛ'}
{}
{'title': 'БАРЫШНИК'}
{}
{'title': 'ДРАЗНИТЬСЯ'}
{}
{'title': 'НЕПРОЛАЗНЫЙ'}
{}
{'title': 'ЧЛЕНСТВО'}
{}
{'title': 'ОТУПЕЛЫЙ'}
{}
{'title': 'ПОДПИТКА'}
{}
{'title': 'РАЗДВОИТЬ'}
{}
{'title': 'ПОДХОД'}
{}
{'ti

In [59]:
dataset.head(10)

,Unnamed: 0,title,text
4186,4186,ГАБАРДИН,Плотная шерстяная ткань с мелкими наклонными р...
23503,23503,РЕНКЛОД,Сорт крупной сладкой сливы. II прил. ренклбдны...
9798,9798,КНУТОВИЩЕ,Рукоятка кнута.
15543,15543,ОДЕКОЛОН,Гигиеническое и освежающее средство - спиртово...
8987,8987,ИТАК,"Следовательно, таким образом. Я, вопрос решен."
25946,25946,СОПУТСТВОВАТЬ,"Идти вместе с кем-чем-н., сопровождать кого-чт..."
24968,24968,СИРОТЕТЬ,"Становиться сиротой. II сов. осиротеть, -ею, -..."
15908,15908,ОПИВКИ,"Остатки от выпитого, недопитое питье."
26742,26742,СТРОЙНЫЙ,"Красиво и правильно сложенный, хорошего телосл..."
20544,20544,ПРИДАВИТЬ,"Надавив, прижать. П. палец дверью. Обвалом при..."
